# Process 2015 Data

Goals:
- Use the 2015 MAUDE data to create a sample dataset for experimentation

Steps:
1. Identify all common data files

|File                    |Description|Required|
|------------------------|-----------|--------|
|`mdrfoithru2021.zip`    |Master Record through 2021|X|
|`patientthru2021.zip`   |Patient Record through 2021|X|
|`foitextchange.zip`     |Narrative data updates: changes to existing narrative data and additional narrative data for existing base records|X|
|`patientproblemcode.zip`|Device Data for patientproblemcode||
|`patientproblemdata.zip`|Patient Problem Data||
|`patientchange.zip`     |MAUDE Patient data updates: changes to existing Base data||
|`mdrfoichange.zip`      |MAUDE Base data updates: changes to existing Base data||
|`devicechange.zip`      |Device data updates: changes to existing Device data and additional Device data for existing Base records||
|`deviceproblemcodes.zip`|Device Problem Data||
|`foidevproblem.zip`     |Device Data for foidevproblem||

2. Identify all 2015 data files

|File                    |Description|Required|
|------------------------|-----------|--------|
|`device2015.zip`        |Device Data for 2015|X|
|`foitext2015.zip`       |Narrative Data for 2015|X|

3. Create databases for each data type
4. Create a merged dataset using joins for each Master Data Record ID in the 2015 data

## Unzip data files
Goals:
- _*(COMPLETE)*_ Unzip downloaded data files to extract the archived `.txt` file

The following Python code completes these steps:
1. Identify the data directory, working directory, and data files
1. Create the working directory if needed
1. Unzip the data files into the working directory

In [28]:
from zipfile import ZipFile
import os

# Identify the data directory, working directory, and data files
data_directory = './data'
working_directory = './2015'
data_files = ['device2015.zip', 'foitextchange.zip', 'patientthru2021.zip',
              'foitext2015.zip', 'mdrfoithru2021.zip']


# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok = True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")

# Unzip the data files into the working directory
for i in data_files:
    print(f"Unzipping {i}")
    with ZipFile(f"{data_directory}/{i}", "r") as zip:
        zip.extractall(f"{working_directory}")

print("Unzip complete.")

Unzipping device2015.zip
Unzipping foitextchange.zip
Unzipping patientthru2021.zip
Unzipping foitext2015.zip
Unzipping mdrfoithru2021.zip


## Create a database for each data file
Goals:
- Read the data files into dataframes
- Write the dataframes to tables in an SQLite database

The following Python code completes these steps:
1. Create a database
1. Process each file in the working directory
1. Get the file path and the base name of the file by removing '.txt'
1. Create a dataframe for the file using Pandas, reading each file as a comma seperated values (TSV) file, using the pipe seperator
1. Set the MDR_REPORT_KEY as the index for the dataframe
1. Write the dataframe to a new SQLite table


In [29]:
import pandas as pd
import os
import csv
import sqlite3


# Create a database
db = sqlite3.connect(f"{working_directory}/database.sqlite3")

# Process each file in the working directory
for root, dirs, files in os.walk(working_directory):
   for file_name in files:

      # Only process '.txt' files...
      if file_name.endswith(".txt"):
         print(f"Processing {file_name}")
         
         # Get the file path and the base name of the file by removing '.txt'
         file_path = os.path.join(root, file_name)
         base_name = file_name.replace('.txt','')

         # Create a dataframe for the file
         df = pd.read_csv(file_path, 
            sep="|", 
            encoding="ISO-8859-1", 
            on_bad_lines='warn', 
            quoting=csv.QUOTE_NONE)

         # Set the MDR_REPORT_KEY as the index for the dataframe
         df.set_index('MDR_REPORT_KEY', inplace=True)
         
         # Write the dataframe to a new SQLite table
         df.to_sql(base_name, db, if_exists="replace")

         # Remove the dataframe in an attempt to free up memory
         del df

print("Database creation complete")

Processing foitextChange.txt
Processing foitext2015.txt
Processing DEVICE2015.txt
Processing patientThru2021.txt
Processing mdrfoiThru2021.txt


b'Skipping line 11971429: expected 82 fields, saw 83\n'
/var/folders/12/zfq9jlbn7pj6kffqfjsrs8fc0000gp/T/ipykernel_98545/4054705136.py:23: DtypeWarning: Columns (0,1,5,6,12,13,16,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,74,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
